In [1]:
!git clone https://github.com/myprivatelearningacc/Autoscaling-Analysis.git

Cloning into 'Autoscaling-Analysis'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 38 (delta 9), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 146.65 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
%cd Autoscaling-Analysis

/content/Autoscaling-Analysis


In [5]:
!ls

notebooks  README.md  src


In [7]:
!mkdir -p data/raw

In [8]:
!gdown 1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk -O data/raw/train.txt

Downloading...
From (original): https://drive.google.com/uc?id=1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk
From (redirected): https://drive.google.com/uc?id=1-c20gJwbzjLsUWFFKbC1OOKxUb17MHSk&confirm=t&uuid=ab2c945e-db64-4da9-bec7-5541708b6897
To: /content/Autoscaling-Analysis/data/raw/train.txt
100% 320M/320M [00:02<00:00, 121MB/s]


In [9]:
!ls data/raw
!wc -l data/raw/train.txt

train.txt
2934960 data/raw/train.txt


In [10]:
from pathlib import Path
from src.preprocessing import load_and_clean_log

DATA_PATH = Path("data/raw/train.txt")

df = load_and_clean_log(DATA_PATH)
df.head()


,host,method,url,status,bytes
time,,,,,
1995-07-01 00:00:01-04:00,199.72.81.55,GET,/history/apollo/,200,6245
1995-07-01 00:00:06-04:00,unicomp6.unicomp.net,GET,/shuttle/countdown/,200,3985
1995-07-01 00:00:09-04:00,199.120.110.21,GET,/shuttle/missions/sts-73/mission-sts-73.html,200,4085
1995-07-01 00:00:11-04:00,burger.letters.com,GET,/shuttle/countdown/liftoff.html,304,0
1995-07-01 00:00:11-04:00,199.120.110.21,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,200,4179


In [16]:
import pandas as pd
df_1m =  pd.DataFrame({
    "requests": df.resample("1min").size(),
    "bytes": df["bytes"].resample("1min").sum(),
    "error_rate": df["status"].ge(400).resample("1min").mean()
})

df_1m.head()

,requests,bytes,error_rate
time,,,
1995-07-01 00:00:00-04:00,42,608453,0.000000
1995-07-01 00:01:00-04:00,61,910128,0.016393
1995-07-01 00:02:00-04:00,57,628556,0.000000
1995-07-01 00:03:00-04:00,71,1747389,0.000000
1995-07-01 00:04:00-04:00,70,1383149,0.000000


In [15]:
df_5m = pd.DataFrame({
    "requests": df.resample("5min").size(),
    "bytes": df["bytes"].resample("5min").sum(),
    "error_rate": df["status"].ge(400).resample("5min").mean()
})

df_5m.head()

,requests,bytes,error_rate
time,,,
1995-07-01 00:00:00-04:00,301,5277675,0.003322
1995-07-01 00:05:00-04:00,267,5041043,0.000000
1995-07-01 00:10:00-04:00,242,6111846,0.016529
1995-07-01 00:15:00-04:00,282,4559748,0.014184
1995-07-01 00:20:00-04:00,319,7262385,0.000000


In [18]:
df_15m = pd.DataFrame({
    "requests": df.resample("15min").size(),
    "bytes": df["bytes"].resample("15min").sum(),
    "error_rate": df["status"].ge(400).resample("15min").mean()
})

df_15m.head()

,requests,bytes,error_rate
time,,,
1995-07-01 00:00:00-04:00,810,16430564,0.006173
1995-07-01 00:15:00-04:00,945,18046060,0.005291
1995-07-01 00:30:00-04:00,1006,22420824,0.003976
1995-07-01 00:45:00-04:00,804,24088038,0.012438
1995-07-01 01:00:00-04:00,750,17178646,0.002667


In [19]:
#Choose df_5m for lag feature
LAGS = [1, 2, 3, 6]  # 5m, 10m, 15m, 30m

for lag in LAGS:
    df_5m[f"requests_lag_{lag}"] = df_5m["requests"].shift(lag)
    df_5m[f"bytes_lag_{lag}"] = df_5m["bytes"].shift(lag)

In [20]:
#Rolling statistic for short-term prediction
ROLL_WINDOWS = [3, 6]

for w in ROLL_WINDOWS:
    df_5m[f"requests_roll_mean_{w}"] = df_5m["requests"].rolling(w).mean()
    df_5m[f"bytes_roll_mean_{w}"] = df_5m["bytes"].rolling(w).mean()


In [21]:
#Calendar
df_5m["hour"] = df_5m.index.hour
df_5m["day_of_week"] = df_5m.index.dayofweek
df_5m["is_weekend"] = (df_5m["day_of_week"] >= 5).astype(int)


In [22]:
df_5m = df_5m.dropna()
df_5m.head()

,requests,bytes,error_rate,requests_lag_1,bytes_lag_1,requests_lag_2,bytes_lag_2,requests_lag_3,bytes_lag_3,requests_lag_6,bytes_lag_6,requests_roll_mean_3,bytes_roll_mean_3,requests_roll_mean_6,bytes_roll_mean_6,hour,day_of_week,is_weekend
time,,,,,,,,,,,,,,,,,,
1995-07-01 00:30:00-04:00,326,9250029,0.000000,344.0,6223927.0,319.0,7262385.0,282.0,4559748.0,301.0,5277675.0,329.666667,7.578780e+06,296.666667,6.408163e+06,0,5,1
1995-07-01 00:35:00-04:00,361,7024223,0.005540,326.0,9250029.0,344.0,6223927.0,319.0,7262385.0,267.0,5041043.0,343.666667,7.499393e+06,312.333333,6.738693e+06,0,5,1
1995-07-01 00:40:00-04:00,319,6146572,0.006270,361.0,7024223.0,326.0,9250029.0,344.0,6223927.0,242.0,6111846.0,335.333333,7.473608e+06,325.166667,6.744481e+06,0,5,1
1995-07-01 00:45:00-04:00,205,7298172,0.039024,319.0,6146572.0,361.0,7024223.0,326.0,9250029.0,282.0,4559748.0,295.000000,6.822989e+06,312.333333,7.200885e+06,0,5,1
1995-07-01 00:50:00-04:00,313,8360541,0.003195,205.0,7298172.0,319.0,6146572.0,361.0,7024223.0,319.0,7262385.0,279.000000,7.268428e+06,311.333333,7.383911e+06,0,5,1
